In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.appName("Linear Regression").getOrCreate()
df = spark.read.csv("preprocessed_datasets/factorized_data.csv", sep = ',', header=True)

ModuleNotFoundError: No module named 'numpy'

### Casting to Number type

In [27]:
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import col

df = df.withColumn("model_year", col("model_year").cast(IntegerType())) \
               .withColumn("milage", col("milage").cast(DoubleType())) \
               .withColumn("engine_capacity", col("engine_capacity").cast(DoubleType())) \
               .withColumn("engine_horsepower", col("engine_horsepower").cast(DoubleType())) \
               .withColumn("brand_numeric", col("brand_numeric").cast(IntegerType())) \
               .withColumn("transmission_numeric", col("transmission_numeric").cast(IntegerType())) \
               .withColumn("fuel_type_numeric", col("fuel_type_numeric").cast(IntegerType())) \
               .withColumn("ext_col_numeric", col("ext_col_numeric").cast(IntegerType())) \
               .withColumn("int_col_numeric", col("int_col_numeric").cast(IntegerType())) \
               .withColumn("accident_numeric", col("accident_numeric").cast(IntegerType())) \
                .withColumn("price", col("price").cast(IntegerType()))

In [28]:
feature_columns = ["model_year", "milage", "engine_capacity", "engine_horsepower", 
                   "brand_numeric", "transmission_numeric", 
                   "fuel_type_numeric", "ext_col_numeric", 
                   "int_col_numeric", "accident_numeric"]

In [29]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [30]:
df_vectorized = assembler.transform(df)

In [31]:
final_data = df_vectorized.select("features", "price")

In [32]:
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

### Linear Regression

In [33]:
# Initialize the model
lr = LinearRegression(featuresCol="features", labelCol="price")

# Train the model32
lr_model = lr.fit(train_data)

# Make predictions
predictions = lr_model.transform(test_data)

# Show predictions
predictions.select("features", "price", "prediction").show()


+--------------------+-----+-------------------+
|            features|price|         prediction|
+--------------------+-----+-------------------+
|(10,[0,1,2,3],[19...| 5899|-29806.738452649675|
|(10,[0,1,2,3],[20...| 9000|  19248.09410543274|
|(10,[0,1,2,3],[20...| 9995| 16047.242627533153|
|(10,[0,1,2,3],[20...|17000| -786.1956876302138|
|(10,[0,1,2,3],[20...| 9999|-16749.703322549583|
|(10,[0,1,2,3],[20...|12990| 10103.933693972882|
|(10,[0,1,2,3],[20...|32850| 20760.860278517008|
|(10,[0,1,2,3],[20...|12500|  2333.629342071945|
|(10,[0,1,2,3],[20...|11995| 20539.506344733527|
|(10,[0,1,2,3],[20...|34000| 18718.784478472546|
|(10,[0,1,2,3],[20...|20997| 18814.789451151388|
|(10,[0,1,2,3],[20...| 7900| 15662.061364857014|
|(10,[0,1,2,3],[20...| 9950|  5096.603448229143|
|(10,[0,1,2,3],[20...| 7450|-1506.2064276058227|
|(10,[0,1,2,3],[20...| 6500|  6704.664631607942|
|(10,[0,1,2,3],[20...|26999| 22429.131730512483|
|(10,[0,1,2,3],[20...| 9000| 17206.768231219612|
|(10,[0,1,2,3],[20..

### Model Evaluation

In [34]:
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 63189.586507458174


In [39]:
lr_model.save(r"prediction_models\LR")

Py4JJavaError: An error occurred while calling o1538.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:410)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withSc

In [47]:
new_data = spark.createDataFrame([(2007, 213000, 1.6, 172.0, 24.0, 0.0, 0.0, 11.0, 2.0, 0.0)], 
                                 feature_columns)
new_vector = assembler.transform(new_data)

prediction = lr_model.transform(new_vector)
prediction.select("features", "prediction").show()


Py4JJavaError: An error occurred while calling o1819.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 19) (Laptok executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.col